# 사용자 선호에 맞는 시 창작 모델

### 0. 환경 설정

In [1]:
!python -m pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


In [2]:
!pip install typing_extensions pydantic openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 945.2/945.2 kB 122.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [openai]2m6/7 [openai]c]


In [3]:
!pip install datasets transformers peft trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 75.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 67.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.4 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 233.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 78.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 73.6 MB/s  0:00:00m0:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 181.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 162.9 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 143.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 171.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25/25 [trl]32m24/25 [trl]sets]e]s]ub]


In [ ]:
import os
import torch

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

device = "cuda" if torch.cuda.is_available() else "cpu"

### 1. 지도학습 (기반모델 Q-LoRA 파인튜닝)

##### (1) 학습용 데이터 준비

In [5]:
import json
from datasets import Dataset

# 데이터 로드 및 Dataset 변환
dataset_path = "./korean_poetry_dataset.json"

with open(dataset_path, "r", encoding="utf-8") as f:
    poem_data = json.load(f)

processed_data = [{"topic": item["text"]["topic"], "poem":item["text"]["poem"]} 
                  for item in poem_data]

train_dataset = Dataset.from_list(processed_data)

In [ ]:
from transformers import AutoTokenizer

model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"
# model_name = 'NCSOFT/Llama-VARCO-8B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
def preprocess_text(sample):
    input_texts = [f"주제: {t}\n시: {p}" for t, p in zip(sample["topic"], sample["poem"])]
    model_inputs = tokenizer(
                        input_texts, 
                        padding="max_length", 
                        max_length=512, 
                        truncation=True
                    )
    
    model_inputs['labels'] = model_inputs["input_ids"].copy()
    pad_token_id = tokenizer.pad_token_id
    model_inputs['labels'] = [
        [(l if l != pad_token_id else -100) for l in label] 
        for label in model_inputs['labels']
    ]
    
    return model_inputs

In [ ]:
train_dataset = train_dataset.map(
    preprocess_text, 
    batched=True, 
    remove_columns=["topic", "poem"]
)

Map:   0%|          | 0/2600 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=None)

##### (2) 파인튜닝 학습 준비

- 양자화 설정 > 모델 로드
- 학습 모드로 전환
- LoRA 학습 설정
- TrainingArguments 설정

In [ ]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model.gradient_checkpointing_enable()
model.config.use_cache = False
model.config.attn_implementation = "flash_attention_2"

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.train()

trainable params: 4,587,520 || all params: 3,217,337,344 || trainable%: 0.1426


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Lin

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./q_lora_poem",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    optim="adamw_bnb_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_417/4145227946.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


##### (3) 학습 진행

In [16]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
100,2.058800
200,1.231300
300,1.135900
400,1.084300
500,1.011400
600,0.962800
700,0.880700
800,0.883500
900,0.925300


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=975, training_loss=1.107200442583133, metrics={'train_runtime': 2022.2313, 'train_samples_per_second': 3.857, 'train_steps_per_second': 0.482, 'total_flos': 6.76516218273792e+16, 'train_loss': 1.107200442583133, 'epoch': 3.0})

### 2. 학습된 모델로 시(응답) 생성

##### (1) 모델 로드

In [ ]:
from transformers import pipeline

qlora_checkpoint = "./q_lora_poem/checkpoint-975"

model = AutoModelForCausalLM.from_pretrained(qlora_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generate_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id,
    batch_size=2
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [19]:
topics = ["바람", "비", "노을", "달빛", "안개", "사랑", "이별", "운명", "기다림", "후회", "추억", "시간", "청춘", "변화", "마지막 순간", "군중", "밤거리", "버스", "인생", "빌딩", "사람들", "거짓말", "욕망", "돈", "권력", "비밀", "죽음", "희망", "동물", "자연", "도시", "바다", "산", "하늘", "별", "꽃", "나무", "강", "바위", "흙", "눈", "빗방울", "눈물", "웃음"]

eval_file = 'rlhf_evaluation_data.json'

try:
    with open(eval_file, "r", encoding="utf-8") as f:
        eval_dataset = json.load(f)
except FileNotFoundError:
    eval_dataset = []

In [ ]:
num_batches = 5
batch_size = 20
total_samples = num_batches * batch_size
generated_samples = len(eval_dataset)

##### (2) 시 생성

In [ ]:
import time
import random
from tqdm import tqdm

def generate_poem_batch():
    batch_data = []

    with tqdm(total=batch_size, desc="<시 생성 중>", leave=False) as t:
        for _ in range(batch_size):
            topic = random.choice(topics)
            input_text = f"주제: {topic}\n시:"

            start_time = time.time()
            poem = generate_pipeline(
                                        input_text,
                                        max_new_tokens=100,
                                        temperature=0.8,
                                        top_p=0.9
                                    )[0]['generated_text']
            end_time = time.time()

            gen_time = end_time - start_time
            batch_data.append({
                "topic": topic,
                "poem": poem,
                "selected": None
            })

            t.update(1)

            global generated_samples
            generated_samples += 1
            complete_rate = (generated_samples / total_samples) * 100
            remaining_time = ((total_samples - generated_samples) * gen_time) / 60

            print(f'\n{generated_samples}/{total_samples}개 완료 ({complete_rate:.2f}%)')
            print(f'- 예상 남은 시간 : {remaining_time:.1f}분')
            print('-' * 50)

    return batch_data

In [ ]:
for _ in tqdm(range(num_batches), desc="<전체 진행 상황>", position=0):
    eval_dataset.extend(generate_poem_batch())

    with open(eval_file, 'w', encoding='utf-8') as f:
        json.dump(eval_dataset, f, ensure_ascii=False, indent=4)

<시 생성 중>:   5%|▌         | 1/20 [00:02<00:56,  3.00s/it]


1/100개 완료 (1.00%)
- 예상 남은 시간 : 4.9분
--------------------------------------------------



<시 생성 중>:  10%|█         | 2/20 [00:05<00:50,  2.82s/it]


2/100개 완료 (2.00%)
- 예상 남은 시간 : 4.4분
--------------------------------------------------



<시 생성 중>:  15%|█▌        | 3/20 [00:08<00:47,  2.77s/it]

...


98/100개 완료 (98.00%)
- 예상 남은 시간 : 0.1분
--------------------------------------------------



<시 생성 중>:  95%|█████████▌| 19/20 [00:51<00:02,  2.73s/it]


99/100개 완료 (99.00%)
- 예상 남은 시간 : 0.0분
--------------------------------------------------



<전체 진행 상황>: 100%|██████████| 5/5 [04:29<00:00, 53.96s/it]    


100/100개 완료 (100.00%)
- 예상 남은 시간 : 0.0분
--------------------------------------------------


##### (3) 피드백
- 생성된 시에 대해 selected="True"로 수정해 피드백 반영

### 3. Reward Model 학습

##### (1) 데이터 로드 및 처리

In [ ]:
with open(eval_file, "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

reward_data = [
    {'text_a': f'주제: {item["topic"]}', 'text_b': item['poem']}
    for item in evaluation_data if item['selected']
]

reward_dataset = Dataset.from_list(reward_data)

In [ ]:
def preprocess_reward_data(sample):    
    model_inputs = tokenizer(
                        sample["text_a"],
                        text_pair=sample["text_b"],
                        padding="max_length", 
                        max_length=512, 
                        truncation=True
                    )
    
    model_inputs['labels'] = model_inputs["input_ids"].copy()
    pad_token_id = tokenizer.pad_token_id
    model_inputs['labels'] = [
        [(l if l != pad_token_id else -100) for l in label] 
        for label in model_inputs['labels']
    ]
    
    return model_inputs

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

reward_dataset = reward_dataset.map(
    preprocess_reward_data,
    batched=True,
    remove_columns=['text_a', 'text_b']
)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

##### (2) 학습 준비

- 양자화 설정 > 모델 로드
- LoRA 학습 설정
- TrainingArguments 설정

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    device_map="auto"
)

In [ ]:
reward_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
)

reward_model = prepare_model_for_kbit_training(reward_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
reward_model = get_peft_model(reward_model, lora_config)

In [ ]:
reward_training_args = TrainingArguments(
    output_dir="./reward_model",
    save_strategy="epoch",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=True
)

reward_trainer = Trainer(
    model=reward_model,
    args=reward_training_args,
    train_dataset=reward_dataset,
    tokenizer=tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_417/3596903846.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  reward_trainer = Trainer(


##### (3) 학습 진행

In [34]:
reward_trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=6, training_loss=3.3420845667521157, metrics={'train_runtime': 9.3087, 'train_samples_per_second': 3.223, 'train_steps_per_second': 0.645, 'total_flos': 260198545489920.0, 'train_loss': 3.3420845667521157, 'epoch': 3.0})

### 4. RLHF (ORPO)

##### (1) 모델 로드

In [ ]:
model = AutoModelForCausalLM.from_pretrained(qlora_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.train()
model.cuda()

for param in model.parameters():
    param.requires_grad = True

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

##### (2) ORPO 데이터셋 준비

In [ ]:
with open(eval_file, "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

orpo_data = []

for item in evaluation_data:
    if item['selected']:
        prompt_text = f'주제: {item["topic"]}\n이 주제에 맞는 시를 작성해 주세요.'
        chosen_text = item['poem']
        rejected_text = ""

        tokenized_prompt = tokenizer(prompt_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")
        tokenized_chosen = tokenizer(chosen_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")
        tokenized_rejected = tokenizer(rejected_text, truncation=True, padding="max_length", max_length=64, return_tensors="pt")

        orpo_data.append({
            "prompt": prompt_text,
            "chosen": chosen_text,
            "rejected": rejected_text,
            "prompt_input_ids": tokenized_prompt['input_ids'].squeeze(0).cuda(),
            "prompt_attention_mask": tokenized_prompt['attention_mask'].squeeze(0).cuda(),
            "chosen_input_ids": tokenized_chosen['input_ids'].squeeze(0).cuda(),
            "chosen_attention_mask": tokenized_chosen['attention_mask'].squeeze(0).cuda(),
            "rejected_input_ids": tokenized_rejected['input_ids'].squeeze(0).cuda(),
            "rejected_attention_mask": tokenized_rejected['attention_mask'].squeeze(0).cuda(),
        })

        orpo_dataset = Dataset.from_list(orpo_data)

##### (3) ORPO 설정

In [ ]:
from trl import ORPOConfig

orpo_config = ORPOConfig(
    output_dir='./orpo_output',
    per_device_train_batch_size=1,
    num_train_epochs=5,
    learning_rate=2e-6,
    gradient_accumulation_steps=4,
    logging_steps=50,
    fp16=False,
    bf16=True,
    remove_unused_columns=False,
    gradient_checkpointing=True,
    max_grad_norm=1.0,
    warmup_steps=100,
    save_steps=500,
    save_total_limit=2
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
from trl.trainer.utils import DPODataCollatorWithPadding

data_collator = DPODataCollatorWithPadding(
    pad_token_id=tokenizer.pad_token_id,
    label_pad_token_id=-100,
    is_encoder_decoder=False
)

In [ ]:
from trl import ORPOTrainer

orpo_trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    train_dataset=orpo_dataset,
    data_collator=data_collator,
    processing_class=tokenizer
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

##### (4) ORPO 적용

In [ ]:
# torch.cuda.empty_cache()
orpo_trainer.train()
# torch.cuda.empty_cache()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128009}.


Step,Training Loss


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### 최종 시 생성

In [42]:
orpo_checkpoint = './orpo_output/checkpoint-15'

model = AutoModelForCausalLM.from_pretrained(orpo_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_name)

generate_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [43]:
import random

def generate_poem_final(num_samples=5):
    topics = ["바람", "비", "노을", "달빛", "안개", "사랑", "이별", "운명", "기다림", "후회", "추억", "시간", "청춘", "변화", "마지막 순간", "군중", "밤거리", "버스", "인생", "빌딩", "사람들", "거짓말", "욕망", "돈", "권력", "비밀", "죽음", "희망", "동물", "자연", "도시", "바다", "산", "하늘", "별", "꽃", "나무", "강", "바위", "흙", "눈", "빗방울", "눈물", "웃음"]
    result = []

    for _ in range(num_samples):
        topic = random.choice(topics)
        input_text = f'주제: {topic}\n시:'
        poem = generate_pipeline(
                                    input_text,
                                    max_new_tokens=100,
                                    temperature=0.8,
                                    top_p=0.9
                                )[0]['generated_text']
        result.append({"topic": topic, "poem": poem})

    return result

In [ ]:
generated_poem = generate_poem_final(num_samples=10)
generated_poem